In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Plate (2D)

alpha = 110
length = 50 # length of the plate, 50 mm
time = 4 # total time, 4 s
x_nodes = 30 # number of nodes - 1
y_nodes = 20

dx = length / x_nodes # distance between nodes
dy = length / y_nodes # distance between nodes
# time step, must be less or equal to than min of dx^2 / (4 * alpha) and dy^2 / (4 * alpha)
dt = np.min([0.25 * dx**2 / alpha, 0.25 * dy**2 / alpha]) 

x_train = np.linspace(0, length, x_nodes)
y_train = np.linspace(0, length, y_nodes)
t_all = np.linspace(0, time, int(time / dt))
train = len(t_all) // 2
t_train = t_all[:train]
t_test = t_all[train:]

u = np.zeros((x_nodes, y_nodes)) + 20 # middle of the plate is 20 degrees
# for i in range(len(u[0, :])): # top ranges from 20 to 78 degrees linearly
#     u[0, i] = 20 + 3*i
u[0, :] = 100 # bottom side of the plate is 100 degrees
# u[-1, :] = 100 # top side of the plate is 100 degrees
# u[:, 0] = 100 # left side of the plate is 100 degrees
# u[:, -1] = 100 # right side of the plate is 100 degrees

heat_data = np.zeros((int(time / dt), x_nodes, y_nodes))

for counter in range(heat_data.shape[0]):
    w = u.copy()

    for i in range(1, x_nodes - 1):
        for j in range(1, y_nodes - 1):
            dd_ux = (w[i - 1, j] - 2 * w[i, j] + w[i + 1, j]) / dx**2
            dd_uy = (w[i, j - 1] - 2 * w[i, j] + w[i, j + 1]) / dy**2

            u[i, j] = dt * alpha * (dd_ux + dd_uy) + w[i, j]

    heat_data[counter, :, :] = u.copy()

    # print(f't: {counter * dt:.3f} s, Ave temp: {np.mean(u):.2f} C')

print(heat_data.shape)

In [ ]:
samp_time = len(t_train) // 2 # plot the time in the middle of training

fig, axis = plt.subplots()
pcm = axis.pcolormesh(y_train, x_train, heat_data[samp_time, :, :], cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
axis.set_title(f'True Data\n t: {samp_time*dt:.3f} s')
plt.xlabel('y (mm)')
plt.ylabel('x (mm)')
plt.show()

In [ ]:
import os
import sys

parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from deep_learning import deep_network_core as core, utils
import torch
import torch.nn as nn
from torch.autograd import grad as autograd

In [ ]:
# torch.manual_seed(1234)
# np.random.seed(1234)

Tr, Xr, Yr = np.meshgrid(t_train, x_train, y_train)

inpt = np.column_stack((
    Tr.transpose(1, 0, 2).ravel(), # Transpose needed to get in t, x, y order
    Xr.transpose(1, 0, 2).ravel(), 
    Yr.transpose(1, 0, 2).ravel(),
    ))
oupt = heat_data[:train, :, :].ravel()

In [ ]:
class MSE_Loss(core.LOSS):
    def __init__(self):
        self.loss = nn.MSELoss()
        
    def __call__(self, target, result, model):
        return self.loss(target, result)

class Huber_Loss(core.LOSS):
    def __init__(self):
        self.loss = nn.HuberLoss()
        
    def __call__(self, target, result, model):
        return self.loss(target, result)

class PHYSICS_Loss(core.LOSS):
    # def __init__(self, bounds):
    #     self.bounds = bounds
    
    def __call__(self, target, result, model):
        x = torch.empty((100, 1)).uniform_(0, length).requires_grad_(True)
        y = torch.empty((100, 1)).uniform_(0, length).requires_grad_(True)
        t = torch.empty((100, 1)).uniform_(0, time).requires_grad_(True)
        inp = torch.cat((x, y, t), axis=1)
        zs = model(inp)
        pde = utils.dy_dt(zs, t) - utils.laplacian_2d(zs, x, y) # dz/dt - (ddz/dxx + ddz/dyy)
        return torch.mean(pde**2)

In [ ]:
x = torch.empty((100, 1)).uniform_(0, length).requires_grad_(True)
y = torch.empty((100, 1)).uniform_(0, length).requires_grad_(True)
t = torch.empty((100, 1)).uniform_(0, time).requires_grad_(True)
inp = torch.cat((x, y, t), axis=1)

inp.shape

In [ ]:
network_nn = core.PINN(3, 1, 64, 3, [(1, MSE_Loss())])

In [ ]:
print("Training NN")
network_nn.fit(inpt, oupt, lr=1e-3, epochs=1000)


In [ ]:
predicted = network_nn.predict(inpt).reshape((len(t_train), len(x_train), len(y_train)))

fig, axis = plt.subplots()
pcm = axis.pcolormesh(y_train, x_train, predicted[samp_time, :, :], cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
axis.set_title(f'Neural Net Trained\n t: {samp_time*dt:.3f} s')
plt.xlabel('y (mm)')
plt.ylabel('x (mm)')
plt.show()

In [ ]:
Tr_test, Xr_test, Yr_test = np.meshgrid(t_test, x_train, y_train)

inpt_test = np.column_stack((
    Tr_test.transpose(1, 0, 2).ravel(), # Transpose needed to get in t, x, y order
    Xr_test.transpose(1, 0, 2).ravel(), 
    Yr_test.transpose(1, 0, 2).ravel(),
    ))

predicted_test = network_nn.predict(inpt_test).reshape((len(t_test), len(x_train), len(y_train)))

fig, axis = plt.subplots()
pcm = axis.pcolormesh(y_train, x_train, predicted_test[samp_time, :, :], cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
axis.set_title(f'Neural Net Extrapolation\n t: {(samp_time+train)*dt:.3f} s')
plt.xlabel('y (mm)')
plt.ylabel('x (mm)')
plt.show()

In [ ]:
network_pinn = core.PINN(3, 1, 64, 3, [(1, MSE_Loss()), (1, PHYSICS_Loss())])

print("Training PINN")
network_pinn.fit(inpt, oupt, lr=1e-3, epochs=1000)

In [ ]:
predicted = network_pinn.predict(inpt).reshape((len(t_train), len(x_train), len(y_train)))

fig, axis = plt.subplots()
pcm = axis.pcolormesh(y_train, x_train, predicted[samp_time, :, :], cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
axis.set_title(f'PINN Trained\n t: {samp_time*dt:.3f} s')
plt.xlabel('y (mm)')
plt.ylabel('x (mm)')
plt.show()

In [ ]:
predicted_test = network_pinn.predict(inpt_test).reshape((len(t_test), len(x_train), len(y_train)))

fig, axis = plt.subplots()
pcm = axis.pcolormesh(y_train, x_train, predicted[samp_time, :, :], cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
axis.set_title(f'PINN Extrapolation\n t: {(samp_time+train)*dt:.3f} s')
plt.xlabel('y (mm)')
plt.ylabel('x (mm)')
plt.show()

In [ ]:
fig, axis = plt.subplots()
pcm = axis.pcolormesh(y_train, x_train, heat_data[(samp_time+train), :, :], cmap='gist_rainbow', vmin=0, vmax=100)
cbar = plt.colorbar(pcm, ax=axis)
cbar.set_label('Temperature (°C)')
axis.set_title(f'True Data\n t: {(samp_time+train)*dt:.3f} s')
plt.xlabel('y (mm)')
plt.ylabel('x (mm)')
plt.show()

In [ ]:
network_dgm = core.DGM(3, 1, 64, 3, [(1, MSE_Loss())])
print("Training DGM")
network_dgm.fit(inpt, oupt, lr=1e-3, epochs=1000)

In [ ]:
network_pi_dgm = core.DGM(3, 1, 64, 3, [(1, MSE_Loss()), (1, PHYSICS_Loss())])
print("Training PI-DGM")
network_pi_dgm.fit(inpt, oupt, lr=1e-3, epochs=1000)